# Practical: Data Analysis

In this assignment you will experiment with

- exploring the data collected at a home and a weather station around the Eindhoven area,
- building a predictive model for estimating the amount of electricity produced by the solar panels at the home given a weather forecast.

This notebook will guide you through the typical steps that such work would involve. It demonstrates how to explore and clean the data, and how to use it to train predictive models.

#### Goal

Your task is to train a model on the time series data containing:

- measurements of the amount of electricity produced by the solar panels on the roof of a home in the Eindhoven area
- weather measurements around the Eindhoven airport

in order to predict the hourly solar panel output given the weather forecast for a particular hour of a particular day.

#### Jupyter notebooks

For those who are new to jupyter notebooks, a typical notebook contains text cells (like this one) interleaved with code cells (the gray boxes). You can execute a cell by selecting it and pressing Shift+Enter. The expression in the last line in a cell is the output of that cell. Try executing the following cell.

In [ ]:
def add(a, b):
    return a + b

add(1,2)

You can find out more about jupyter notbooks at:

- https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/
- http://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Notebook%20Basics.ipynb
- http://nbviewer.jupyter.org/github/jupyter/notebook/tree/master/docs/source/examples/Notebook/

#### References

The "Python Data Scinence Handbook" provides a good overview of using python for data data analysis and you are encouraged to consult it during this assignment:

- https://jakevdp.github.io/PythonDataScienceHandbook/

The "Introduction to Statistical Learning" [ISLR] (7th edition) book provides a good introduction to machine learning from the statistical perspective:

- http://faculty.marshall.usc.edu/gareth-james/ISL/

The "Dataset Shift in Machine Learning" book by Joaquin Quiñonero-Candela, Masashi Sugiyama, Anton Schwaighofer and Neil D. Lawrence provides a good overview of the problems that may arise when the test and training inputs and outputs have different distributions. You can find more information about the book at:

- https://mitpress.mit.edu/books/dataset-shift-machine-learning

#### Deliverable

Throughout this notebook you will find cells starting with `#TODO` and `#BEGIN_TODO`. Fill in all these TODO cells. The `#TODO` cells are meant to guide you, while the `#BEGIN_TODO` _answer cells_ will be graded.

- Answer cells start and end with tags, `# // BEGIN_TODO [Q0]` and `# // END_TODO [Q0]`, for example. Do not edit these tags in any way, else your answers may not be parsed by our grading system. 
- Your answers should be provided in Python code format between the tag lines, for example:
```
# // BEGIN_TODO [Q0]
q0 = 1
# // END_TODO [Q0]
```
- Do not place any other code between the tags, unless explicitly requested.
- Do not reuse the variable names from the answer cells that you are supposed to use to fill in answers, so in case of the example above, do not reuse `q0` later in the notebook.

You are encouraged to play with the data and extend this notebook in order to obtain your answers. You may insert cells at any point in the notebook, as long as the question cells remain unaltered. At the end, deliver the filled in `.ipynb` file.

> **IMPORTANT:** Before delivering your notebook, make sure that the cells in your notebook can be executed in sequence without errors, by executing "Restart & Run All" from the "Kernel" menu. This is also how your notebook will be evaluated.

Let's get started by filling in your details in the following answer cell. Assign your names and student ids to variables `name_student1`, `id_student1`, `name_student2`, `id_student2`, e.g.:

```
# // BEGIN_TODO [Q0]
name_student1 = "John Smith"
id_student1 = "1234567"
name_student2 = "Jane Miller"
id_student2 = "7654321"
# // END_TODO [Q0]
```

In [ ]:
#// BEGIN_TODO [Q0]
#// END_TODO [Q0]

### Import the relevant libraries

In this assignment we will be using mainly the following libraries:

- `pandas` for organizing the data
- `numpy` for operating on the data
- `matplotlib` for visualizing the data
- `sklearn` for training and evaluating a model on the data
- other utility libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.model_selection import *
import glob

### Read the data

The data resides in two files:

In [ ]:
raw_energy = pd.read_csv("energy_171819.csv")
raw_energy[95500:956000]

In [ ]:
raw_weather = pd.read_csv("weather_171819.csv")
raw_weather.head()

When analysing data it is important to understand its semantics. In IoT one needs to be extra careful, especially when integrating data from various sources. The very basic information are the units of the measurements, but also the specifications of the sensors gathering the data and the processes that are being monitored. For example, in this case the specification of the solar panel installation states that it can generate max 153k Jouls per minute.  

Assume the following for the energy data:
- `seconds` is the time the sample was recorded (UTC) as an epoch timestamp
- `total_consumption` is the total amount of electricity that was pulled from the grid (kWh)
- `total_production` is the total amount of electricity that was pushed into the grid (kWh)
- `solar_production` is the amount of eletricity that was produced since the last sample (Wh).
- `total_solar_production` is the total amount of electricity produced by the solar panels (kWh)

Assume the following for the weather data:
- `time` is the time the sample was recorded (UTC)
- `clouds` measures whether it was cloudy at the time when the sample was taken, ranging from 0-3, 0 meaning no clouds and 3 meaning very cloudy.
- `temperature` is the still temperature (C).
- `wind direction` given in South (Z), West (W), North (N), and (East), or a combination thereof.
- `wind speed` in m/s. 
- `visibility` in meters.
- `air pressure` in hPa.

### Explore the data

Exploration is usually the first step in any data analysis task. Visualization is an important tool for exploring the data. It gives insights into the structure and semantics of the data and indications for how to clean it.

The `matplotlib` library provides a collection of useful plots, such as a line plot, scatter plot, histogram, scatter mattrix, etc. You can find out more about this library at 

- https://matplotlib.org/users/pyplot_tutorial.html
- https://matplotlib.org/devdocs/gallery/

The `pandas` library also contains convenient wrappers around the `matplotlib` library for visualizing data frames and series:

- https://pandas.pydata.org/pandas-docs/stable/visualization.html

Let's draw a simple plot of the energy data (note that we're only plotting a small part of the data here):

In [ ]:
plt.rcParams["figure.figsize"] = (14, 9) # (w, h)
X = raw_energy[:300000]["seconds"]
Y = raw_energy[:300000]["total_solar_production"]
plt.plot(X, Y)
plt.xlabel('Epoch time (s)')
plt.ylabel('Total solar production (kWh)')
plt.axis([X.min(), X.max(), Y.min(), Y.max()])

In [ ]:
# TODO: explore the values in different columns (experiment with different visualizations, e.g. line, histogram, ...)

### Question 1

Simple visual inspection of the data can already tell you a lot about what you are working with. The energy data covers 2017 , 2018, and 2019, but because the data was split sometime beforehand, some time ranges will be missing.

Assign to `question_missing_months` in the form of a list the months that are missing from the energy data (for example, if June and July of 2017 are missing, the list would be `["2017-06", "2017-07"]`). 

Not only are there months missing, each month that is in the energy data has some missing days as well. Assign to `question_num_missing_days` the number of days that is missing from each month.

You will have to convert the timestamps in the data to retrieve the months and days. Note that the Netherlands is in timezone UTC+1.

In [ ]:
#// BEGIN_TODO [Q1]
#// END_TODO [Q1]

### Clean the data

Data cleaning is an important part of any data analysis task. According to the general wisdom, most effort actually is spent on data cleaning. It involves preparing the data for the following steps, e.g. converting values to the right type, filling in missing values, removing outliers, normalizing the data, etc.

In the plot above you may have noticed vertical lines (you may have to plot a shorter time range to spot them). These lines correspond to 0 values and are likely to be measurement errors (e.g. the solar panel meter has crashed). Such outliers will often negatively impact the accuracy of the predictive model. However, care must be taken to properly define what constitutes an outlier and what constitutes a valid measurement. Some odd looking measurements may actually contain important evidence for proving or disproving a hypothesis. In this case the 0 values can be considered measurement errors and should be removed.

Pandas provides convenient methods for selecting subsets of the data that can be used for removing outliers:

- https://jakevdp.github.io/PythonDataScienceHandbook/03.02-data-indexing-and-selection.html

In [ ]:
# TODO: remove the samples with production total of 0

Data that is collected in the wild will sometimes contain formatting inconsistencies. It is a good habit to check the data types:

In [ ]:
raw_energy.dtypes

As expected, the `'total_consumption'`, `'total_production'`, `'total_solar_production'` columns are of numerical type, however, the `solar_production` is of an `object` type (which is also used for strings), while looking at the specification of the column above we would expect a numerical type. Modeling this column later will require a numerical column, so it will need to be converted to the right type first.

In [ ]:
# TODO: explore the offending values in the solar_production column

### Question 2

Convert empty strings in the `'solar_production'` column to `np.nan`. Convert the remaining values in the `'solar_production'` column to `float` type, and copy the resulting column to a variable named `solar_production_column`.

In [ ]:
#// BEGIN_TODO [Q2]
#// END_TODO [Q2]

### Engineer features

The goal of this assignment is to predict the hourly energy output of the solar panels given the weather forecast. This requires to relate the energy samples with the weather samples, basically attaching the energy production label to the weather measurements. However, the *total* energy and weather samples are taken approximately every 10 seconds and 10 minutes, respectively.

####  Aggregate per hour

We can use pandas grouping functions to aggregate the samples per hour, taking the largest  measurement in that hour.

In [ ]:
col = "total_solar_production"
energy = raw_energy.copy()
times = pd.to_datetime(energy["seconds"], unit='s')

energy = energy.groupby([times.dt.year, times.dt.month, times.dt.dayofyear, times.dt.hour])[[col]].agg(np.max)
energy.index.names = ["year", "month", "day", "hour"]

### Question 3 ###

Note that the hourly aggregation using the maximum works well for columns containing the totals. Does it also work for the `'solar_production'` column?

1. Yes, but the aggregation method needs to be changed to fit the solar_production data type
2. Yes, the same aggregation method can be applied
3. No, it's not possible to aggegrate over the solar_production field
4. No, aggregating over the solar_production field in any way does not make sense.
5. None of the above.

Assign an integer corresponding to the correct answer to variable `question_aggregation`.

In [ ]:
#// BEGIN_TODO [Q3]
#// END_TODO [Q3]

After aggregation, some cells can contain `Nan` values (e.g. when trying to compute a maximum for an hour during which no samples were recorded). Therefore, after performing such operations one should decide what to do with any NaN values. In this case we chose to remove such samples.

In [ ]:
energy = energy.dropna()

After grouping, merging and slicing operations, the index of a data frame might need to be recomputed, to reflect the new ordering of rows.

In [ ]:
energy = energy.reset_index()

#### Compute the hourly production

We are interested in predicting the solar panel output in a given hour. However, the `'total_solar_production'` column in the energy data frame at this point contains the total solar panel output until the end of that hour. Assuming that the data set contains a sample for every hour, i.e. there are no gaps, the hourly output can be compouted by taking the difference between the previous sample.

In [ ]:
energy["production"] = energy[col] - energy[col].shift(1)
energy = energy.dropna().reset_index()
energy.head()

### Question 4 ###

After creating the production column there are NaNs in the data. Where do they come from, and why is the index recomputed?

1. The NaN values are caused by missing values in the original data due to sensor errors, and the index is reset because a column has been added.
2. The NaN values are caused by missing values in the original data due to the way the column is calculated, and the index is reset because a column has been added.
3. The NaN values are caused by missing values in the original data due to sensor errors, and the index is reset because the number of rows has changed.
4. The NaN values are caused by missing values in the original data due to the way the column is calculated, and the index is reset because the number of rows has changed.

Assign the integer corresponding to the correct answer to the variable `question_production`.

In [ ]:
#// BEGIN_TODO [Q4]
#// END_TODO [Q4]

Now we can perform a similar aggregation for the `clouds` column in the weather data. Note that rather than choosing the maximum, we take the average `clouds` value per hour.

In [ ]:
weather = raw_weather.copy()
times = pd.to_datetime(weather["time"])
weather = weather.groupby([times.dt.year, times.dt.month, times.dt.dayofyear, times.dt.hour])["clouds"].agg(np.mean)
weather.index.names = ["year", "month", "day", "hour"]
weather = weather.reset_index()

#### Merge the data frames

Now that both data frames are expressed in terms of hours and the energy data frame contains the hourly solar panel output, we can use pandas to merge them.

**Important:** when dealing with times in data from various sources one must be very carefull about the semantics of the time, such as the time-zone or the daylight-saving. Luckily, in this assignment both energy and weather data use UTC time, so no extra preprocessing is required.

In [ ]:
data = pd.merge(energy, weather).dropna().reset_index(drop=True)
data.head()

#### Explore the engineered features

After engineering features it is a good idea to explore the data set again, to see if extra cleaning is required.

In [ ]:
values = data["production"].values
plt.figure(figsize=(10, 2))
plt.hist(values, 50);
plt.xlabel('Production (kWh)')
plt.ylabel('Frequency')
print (min(values), max(values))

The small variance of the values and their large range suggests there are some outliers. This can be also visualized in a different way.

In [ ]:
def plot(df, col):
    x = df["day"]
    plt.figure(figsize=(10,3))
    plt.plot(x, df[col], 'bo', markersize=1)
    plt.xlabel('Day')
    plt.ylabel(col)

plot(data, "production")

The figure suggests that there are indeed outliers in the energy production data.

### Question 5

The plot shows some outliers. How do you explain these? Note that there may be more than one correct anwser.

1. There is actually an oversight in the previous calculations which causes the outliers.
2. There is some unclean data present due to sensor errors.
3. There were some very clear and sunny days.
4. The outliers represent multiple days of production.

Assign the integer(s) corresponding to the correct answer(s) as a list to the variable `question_outliers`.

In [ ]:
#// BEGIN_TODO [Q5]
#// END_TODO [Q5]

Note that when removing the energy samples containing 0 values in `raw_energy` earlier, it was clear how to define an outlier. In this case, however, one must carefully decide how to define an outlier, to avoid introducing bias. You can read more about outlier detection here:

- Section 3.3.3 of "Introduction to Statistical Learning"
- http://scikit-learn.org/stable/modules/outlier_detection.html

### Question 6

How do you best define an outlier for this data set?

1. Calculate the average production, and define an outlier based on standard deviations.
2. Build a model for the expected production, and label outliers based on the model predictions.
3. Set a fixed threshold for outliers based on the characteristics of the solar panels.
4. Estimate a fixed threshold for outliers by hand based on the above plot.

Assign an integer corresponding to the correct answer to variable `question_outlier_definition`.

In [ ]:
#// BEGIN_TODO [Q6]
#// END_TODO [Q6]

### Question 7

Remove the outliers from the `data` data frame. Assign the resulting data frame to variable `data_no_outliers`. 

In [ ]:
# TODO: remove outliers

In [ ]:
#// BEGIN_TODO [Q7]
#// END_TODO [Q7]

Data visualization is often helpful to check if the results of our data processing and analysis meet our expectations.

In [ ]:
# TODO: plot the clouds feature

### Question 8

Now that the data has been cleaned and converted, let's take another look. Aggregate the months that are present in the training data, and determine which month of the year is on average the most productive in terms of solar production across the 3 years. Assign the integer corresponding to the month to `most_productive_month`.

In [ ]:
#// BEGIN_TODO [Q8]
#// END_TODO [Q8]

### Train a model

For training and evaluating the performance of a model we need to split the data into a training and a test.

In [ ]:
X = data_no_outliers[["clouds"]].values
y = data_no_outliers["production"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

### Evaluate the model

After a model is trained we would like to estimate how well it is performing, whether we can actually trust its predictions if it were deployed. Model performance can be estimated using various visualisations and statistics (see Section 3.1.3 of [ISLR])

#### Model fit

If the feature space is 1 or 2 dimensional, then we can easily plot the model to illustrate how well it fits the test data.

In [ ]:
x_plot = np.linspace(min(X), max(X), 100)
X_plot = x_plot.reshape(-1, 1)
y_plot = model.predict(X_plot)

dots, = plt.plot(X_test, y_test, 'bo', markersize=2, color="red", label="training data");
line, = plt.plot(x_plot, y_plot, linewidth=2, label="model");
plt.xlabel('Clouds')
plt.ylabel('Production (kWh)')
plt.legend(handles=[dots, line]);

We can compute the R2 score to assign a number to how well the model fits the data. It estimates how much of the variance in the data is explained by the model, i.e. how well the model fits the data. A score of 1 means the model captures all of the variance, and a score of 0 means the model does not capture any of the variance.

You can read more about the R2 score in Section 3.1.3 of "Introduction to Statistical Learning".

In [ ]:
model.score(X_test, y_test)

#### Residuals

We can also plot a histogram of the residuals (or errors), i.e. the deviations of the values predicted by our model from the ground truth values (from the test set):

In [ ]:
y_pred = model.predict(X_test)
plt.title("Distribution of the residuals")
plt.hist(y_test - y_pred, 50);
plt.xlabel('Residual')
plt.ylabel('Frequency')

Looking at the histogram you will notice that distribution of the residuals is not symmetric.

### Question 9

Does the model tend to over-estimate, or under-estimate the solar panel output?

1. The model tends to under-estimate.
2. The model tends to over-estimate.
3. The model is not biased

Assign an integer corresponding to the correct answer to variable `question_bias`. 

In [ ]:
#// BEGIN_TODO [Q9]
#// END_TODO [Q9]

If the model is not biased, or if we are not interested in the bias, then we can compute the mean absolute deviation (MAD) to estimate the expected error of a prediction. E.g. a MAD = 0.345 means that we expect on average our predition to be off by 0.345kWh.

In [ ]:
y_pred = model.predict(X_test)
mean_absolute_error(y_test, y_pred)

### Improve the model

We have shown here a simple linear model mapping the `clouds` feature to the `hourly_production` label. However, this model does not perform very well. The problem could be that the model is underfitting or overfitting the data. You can find more about under- and overfitting in Section 2.1.2, 2.1.3, 6.1 of "Introduction to Statistical Learning".

### Question 10 ###

Is the model underfitting or overfitting? What makes you say that? Investigate and then explain in at most 100 words why you think the model is underfitting or overfitting. Assign the explanation to the variable `question_fitting`.

In [ ]:
#// BEGIN_TODO [Q10]
#// END_TODO [Q10]

One could try several things to improve the model.

#### Add more features

The simple linear model assumed that there is a strong relation ship between the clouds (or their absence) and the output of the solar panels. However, clouds alone are not a sufficient indicator, e.g. during the night. You may plot a histogram showing how the `clouds` values are distributed during the 24 hours of a day:

In [ ]:
plt.hist2d(weather.dropna().hour.values, weather.dropna().clouds.values);
plt.xlabel('Hour')
plt.ylabel('Clouds')

### Question 11

Why is the `clouds` column alone not a good indicator for the solar panel output?

Answer this question in max. 100 words and assign your string answer to the variable `question_clouds`.

In [ ]:
#// BEGIN_TODO [Q11]
#// END_TODO [Q11]

While the `clouds` column alone may not be sufficient for predicting the solar planel output, it can be used in combination with other features. Therefore, one way of improving the simple linear model could be to extend it with more features. You can gain insight into which features could be relevant by plotting the relationship between the features and the labels.

In [ ]:
# TODO: use the scatter matrix graph to explore the relationship between the features

### Question 12 ###

Which features are most promising for predicting the solar panel output? Select the two features that are most strongly correlated with production.

- `'year'`
- `'month'`
- `'day'`
- `'hour'`
- `'air pressure'`
- `'visibility'`
- `'wind speed'`
- `'temperature'`

Assign a list of strings corresponding to the promising features to variable `question_features`.

In [ ]:
#// BEGIN_TODO [Q12]
#// END_TODO [Q12]

Note that using additional features may require aggregating them per hour. The `clouds` feature was aggregated taking the maximum value within a given hour, but this may not be suitable for all features.

### Question 13 ###

Using your new-found knowledge of the features, re-define the data on which you train and evaluate your model. Try to improve your model as much as possible by only changing the selection of features.

Assign your new data, new model and the corresponding MAD score to the variables `improved_data`, `improved_model`, `improved_mad`, respectively.

In [ ]:
# TODO: improve model

In [ ]:
#// BEGIN_TODO [Q13]
#// END_TODO [Q13]

#### Choose a more or less flexible model

Another way is to add flexibility to the model e.g. by changing it to a polynomial model or a neural network. You can read more about simple extensions of the linear model in Section 3.3.2 and 6.2 of "Introduction to Statistical Learning".

Linear regression is far from the only modeling we can do here. Another model we could use is that of a Decision Tree. What is nice about these types of models is that they show the exact sequence of decisions that a model makes in order to arrive at a prediction. You can read more about Decision Trees at [https://scikit-learn.org/stable/modules/tree.html]. Note that in this instance, we will want to use the Decision Tree regression model, not the classification model.

### Question 14

The Decision Tree model has quite a number of parameters, and we encourage you to do so to see their impact on the model's performance. One of the most important ones is that of tree depth. What happens with an increase in depth? Note that there may be more than one correct answer.

1. An increase in depth helps to avoid overfitting.
2. An increase in depth helps to avoid underfitting.
3. An increase in depth makes the model more biased towards new data.
4. An increase in depth makes the model less biased towards new data.

Assign the integer(s) corresponding to the correct answer(s) as a list to the variable `decision_tree_depth`. 

In [ ]:
#// BEGIN_TODO [Q14]
#// END_TODO [Q14]

### Question 15

Train a Decision Tree model an assign it to the variable `decision_tree`. Inspect it and determine what the three most influential features are according to this model. Assign the names of these features as a list of strings to the variable `decision_tree_features`. You may want to plot the resulting model to help you inspect it. 

In [ ]:
# TODO: train and inspect decision tree

In [ ]:
#// BEGIN_TODO [Q15]
#// END_TODO [Q15]

### Simulate the forecasting

The `weather_171819_test.csv` file contains the weather data that was held out from `weather_171819.csv` used for training the solar panel output predictor. This data simulates the weather forcasts and is used for evaluating how your model would perform if it was deployed.

In [ ]:
weather_test = pd.read_csv('./weather_171819_test.csv')
weather_test.head()

Provide the predictions for the hourly solar panel output for the given weather forecasts, i.e. populate the `production` column in the following data frame:

In [ ]:
prediction = weather_test.copy()
times = pd.to_datetime(prediction["time"])
prediction['month'] = times.dt.month
prediction['day'] = times.dt.dayofyear
prediction['hour'] = times.dt.hour
prediction = prediction.drop(prediction[prediction['hour'] == 0].index)
prediction = prediction[['month', 'day', 'hour']]
prediction = prediction.drop_duplicates().reset_index(drop=True)
prediction['production'] = np.nan
prediction.head()

**Important:** your model will be evaluated on the held out test set. It is therefore safe to train your *final* model on the entire data set that was provided (rather than the X_train, y_train subsets from the train/test split above), to captures all of the information that is available to you.

### Question 16

Fill in the `'production'` column in the prediction data frame with your predictions (do not reorder the rows). Assign the values from the `'production`' column as a Python list to the variable `forecast`. You may use any predictive model from the `sklearn` package (please do not use any packages that are not already included in this notebook). You can take a look through the contents of `sklearn` at [https://scikit-learn.org/stable/supervised_learning.html] to find supervised models that suit this purpose, such as Random Forests and Multi-layer Perceptrons. 

In [ ]:
# TODO: forecast

In [ ]:
#// BEGIN_TODO [Q16]
#// END_TODO [Q16]

For automating the evaluation of your predictions, call the evaluate function that is defined in the `evaluate.py` script (do not edit the code lines below!):

In [ ]:
from evaluate_stub import *
evaluate(prediction)

As a sanity check, make sure that the `prediction` data frame contains a `production` column and 1919 rows.

Your solution will be evaluated based on the mean absolute error metric.

### Reliability

We often assume that the trained model will operate on data from the same distribution as the data it was trained on (see Chapter 1 of the book "Dataset Shift in Machine Learning"). However, sometimes these assumptions are not true.

Using the model you trained for question 13, evaluate the model's performance on the data set provided below. It contains the energy and weather data for part of the year 2020. Note that as before, this is raw data. You will need to do some cleaning.

In [ ]:
raw_energy_20 = pd.read_csv('./energy_20.csv')
raw_weather_20 = pd.read_csv('./weather_20.csv')

In [ ]:
# TODO: Evaluate the model from previous question on the new data

### Question 17

Investigate and then explain in at most 100 words in the variable `question_change` what you think may be responsible for the change in your model's performance.

In [ ]:
# TODO: Investigate changes

In [ ]:
#// BEGIN_TODO [Q17]
#// END_TODO [Q17]

### Question 18

To make your model robust to these kind of changes, you may want to re-train it on more appropriate data. Using all the data you have been provided so far, compose more suitable training and validation data sets, and assign them to `new_training_data` and `new_validation_data`, respectively. the Scikit-Learn library has some potentially useful functions for generating these data sets (https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection).

In [ ]:
#// BEGIN_TODO [Q18]
#// END_TODO [Q18]

### Predicting consumption

Besides solar production, we are also interested in predicting energy consumption. Being able to accurately predict the power bill may save us from any surprises in the future. The year 2020 has been rather different from most other years, to say the least. Now that you have the data for part of the year 2020, let's look into energy consumption anomalies that may be found here.

### Question 19

The energy consumption shows two dips in the data from 2020 due to the turning off of large electrical appliances. For each of the dips, fill in the start and end day when the decrease occurred in the variables `consumption_dip_1` and `consumption_dip_2` as a list of two dates each. Please use the YYYY-MM-DD format (e.g. `["2020-02-12", "2020-02-14"]`).

In [ ]:
#// BEGIN_TODO [Q19]
#// END_TODO [Q19]

### Question 20

Consumption may depend on other factors than production, so we may need to reconsider the features we chose in question 12. Considering the new training target, which two features from question 12 are most promising for the prediction of energy consumption? Assign them to `question_consumption_features`.

In [ ]:
#// BEGIN_TODO [Q20]
#// END_TODO [Q20]

Train and evaluate your new model on the data from 2017 through 2019 (do not include the data from 2020) so you can predict the energy consumption.

In [ ]:
# TODO: Train a new model to predict energy consumption

### Question 21

As in question 17, check this new model's performance when predicting the energy consumption for the data from 2020. Is there a shift taking place in the data underlying the model? If so, what kind of shift do you think it is? Assign the integer corresponding to the correct answer to `question_consumption_shift`.

1. Sample covariate shift
2. Source component shift

In [ ]:
#// BEGIN_TODO [Q21]
#// END_TODO [Q21]

# Feedback

Please fill in this questionaire to help us improve this course for the next year. Your feedback will be anonymized and will not affect your grade in any way!

### How many hours did you spend on these Exercises?

Assign a number to `feedback_time`.

In [ ]:
#// BEGIN_TODO [Feedback_1]
#// END_TODO [Feedback_1]

### How difficult did you find these Exercises?

Assign an integer to `feedback_difficulty`, on a scale 1 - 10, with 1 being very easy.

In [ ]:
#// BEGIN_TODO [Feedback_2]
#// END_TODO [Feedback_2]

### Did you take the course Foundations of Data Mining?

Assign a boolean value to `feedback_foundations` (True or False).

In [ ]:
#// BEGIN_TODO [Feedback_3]
#// END_TODO [Feedback_3]

### Did you do any other Machine Learning-related coursework (university/workshop/online/etc)

Assign an answer to `feedback_other`, describing any other coursework done.

In [ ]:
#// BEGIN_TODO [Feedback_4]
#// END_TODO [Feedback_4]

### (Optional) Which parts of the assignment did you like?

Assign a string to `feedback_like`.

In [ ]:
#// BEGIN_TODO [Feedback_5]
#// END_TODO [Feedback_5]

### (Optional) Which parts of the assignment could be improved?

Assign a string to `feedback_improve`.

In [ ]:
#// BEGIN_TODO [Feedback_6]
#// END_TODO [Feedback_6]